# 数据预处理

In [1]:
# 导入库
import pandas as pd
import numpy as np

# 加载数据
file = "data/data.xlsx"
data = pd.read_excel(file, header=0)

In [2]:
# 打印数据的前几行
print(data.head()) 

# 去除无关属性
# 截取数据的第7列至“总通最大值”
data = data.iloc[:, 6:-3]

# 打印数据的前几行
print(data.head()) 

   序号  类别        检测日期     牌别  班次  机号  重量平均值    重量相对偏差  重量标准偏差  重量变异系数  ...  \
0  1H  一类  2024-01-02  贵烟(萃)  乙班  5#  0.516  0.006524   0.011   2.202  ...   
1  2H  一类  2024-01-02  贵烟(萃)  甲班  5#  0.515  0.004573   0.012   2.350  ...   
2  3H  一类  2024-01-03  贵烟(萃)  甲班  7#  0.507  0.011032   0.012   2.316  ...   
3  4H  一类  2024-01-03  贵烟(萃)  乙班  7#  0.507  0.011032   0.013   2.549  ...   
4  5H  一类  2024-01-04  贵烟(萃)  甲班  9#  0.509  0.007131   0.009   1.863  ...   

   嘴通最小值  嘴通最大值  总通平均值  总通标准偏差  总通变异系数  总通最小值  总通最大值     水分   含末率  端部落丝量  
0   33.7   41.4   44.6   2.006   4.502   40.1   48.0  12.08  3.26    0.7  
1   31.3   39.6   43.0   2.106   4.904   37.8   47.0  12.12  3.09    0.7  
2   30.9   37.4   42.0   1.810   4.308   38.4   45.3  11.86  4.32    1.2  
3   31.4   41.4   42.8   2.027   4.739   36.8   47.5  11.64  3.69    1.3  
4   33.1   39.9   42.0   1.577   3.751   38.4   45.6  11.63  4.33    2.1  

[5 rows x 49 columns]
   重量平均值    重量相对偏差  重量标准偏差  重量变异系数  重量最小值  重量最大值  吸阻平均值   

In [3]:
# 检查并处理缺失值（NaNs）和异常值（outliers）
def preprocess_data(df):
    # 检查缺失值
    if df.isnull().values.any():
        # 如果存在缺失值，打印缺失值的索引
        print("缺失值的索引：")
        print(df[df.isnull().any(axis=1)].index.tolist())

        print("数据中存在缺失值，正在处理...")

        df = df.fillna(df.mean())  # 用均值填充缺失值
        print("缺失值已处理。")
    else:
        print("数据中没有缺失值。")

    # 检查异常值
    for column in df.select_dtypes(include=[np.number]).columns:
        if (df[column] < 0).any():
            print(f"列 {column} 存在异常值（负数），正在处理...")
            df[column] = df[column].clip(lower=0)

    print("异常值已处理。")
    return df

data = preprocess_data(data)

缺失值的索引：
[592, 593, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767]
数据中存在缺失值，正在处理...
缺失值已处理。
异常值已处理。


In [ ]:
# 特征二元化

# 提取第7列（索引为6）作为R
R = data.iloc[:, 6]
print(R.head())

# 如果 R < 1.27 或者 R > 1.39, 则 R = 0，否则 R = 1
R = np.where((R < 1.27) | (R > 1.39), 0, 1)
print(type(R))
print(R.shape)

# 将R转变为DataFrame格式
R = pd.DataFrame(R, columns=["合格"])
print(R.head())

0    1.275
1    1.321
2    1.291
3    1.296
4    1.350
Name: 吸阻平均值, dtype: float64
<class 'numpy.ndarray'>
(1237,)
   合格
0   1
1   1
2   1
3   1
4   1


In [12]:
# 独热编码

# 查找data列名中是否有关于“硬度”的值
hardness_cols = [col for col in data.columns if "硬度" in col]
print("包含'硬度'的列名：", hardness_cols)

包含'硬度'的列名： ['硬度平均值', '硬度相对偏差', '硬度标准偏差', '硬度变异系数', '硬度最小值', '硬度最大值']


In [16]:
# 将'硬度平均值'一列赋值给hardness
hardness = data.loc[:, "硬度平均值"]
print(hardness.head())

# 如果硬度 < 57，则标记为 [1, 0, 0]; 如果 57 <= 硬度 <= 60，则标记为 [0, 1, 0]; 如果 硬度 > 60，则标记为 [0, 0, 1]
hardness_labels = np.zeros((len(hardness), 3), dtype=int)
hardness_labels[hardness < 57, 0] = 1
hardness_labels[(hardness >= 57) & (hardness <= 60), 1] = 1
hardness_labels[hardness > 60, 2] = 1
print(hardness_labels.shape)
print(type(hardness_labels))

# 将hardness_labels转变为DataFrame格式
hardness_labels = pd.DataFrame(hardness_labels, columns=["硬度_低", "硬度_中", "硬度_高"])
print(hardness_labels.head())

0    56.8
1    58.2
2    61.3
3    62.1
4    57.6
Name: 硬度平均值, dtype: float64
(1237, 3)
<class 'numpy.ndarray'>
   硬度_低  硬度_中  硬度_高
0     1     0     0
1     0     1     0
2     0     0     1
3     0     0     1
4     0     1     0


In [18]:
# 对数据进行归一化处理
print("原始数据的统计信息:")
print(data.describe())

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# 使用 .fit_transform() 一步完成拟合和转换
# 注意：StandardScaler的输出是一个NumPy数组，而不是Pandas DataFrame
data_scaled = scaler.fit_transform(data)

# 为了方便查看，可以将其转换回DataFrame
data_scaled = pd.DataFrame(data_scaled, columns=data.columns)

print("\n-------------------------------------------------")
print("\n标准化后数据的统计信息:")
print(data_scaled.describe())

原始数据的统计信息:
             重量平均值       重量相对偏差       重量标准偏差       重量变异系数        重量最小值  \
count  1237.000000  1237.000000  1237.000000  1237.000000  1237.000000   
mean      0.512656     0.007065     0.010463     2.044025     0.490781   
std       0.004634     0.005635     0.001535     0.294834     0.006928   
min       0.501000     0.000087     0.005000     1.006000     0.466000   
25%       0.509500     0.002623     0.009000     1.838300     0.486000   
50%       0.512000     0.005744     0.010300     2.044000     0.491000   
75%       0.515200     0.010425     0.011200     2.222000     0.495000   
max       0.531000     0.035783     0.017100     3.347000     0.518000   

             重量最大值        吸阻平均值       吸阻相对偏差       吸阻标准偏差       吸阻变异系数  ...  \
count  1237.000000  1237.000000  1237.000000  1237.000000  1237.000000  ...   
mean      0.534255     1.330071     0.019747     0.049897     3.750157  ...   
std       0.007185     0.034055     0.016288     0.008702     0.636770  ...   
min   

In [19]:
# 对数据进行规范化处理
print("原始数据的统计信息:")
print(data.describe())

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# 使用 .fit_transform() 一步完成拟合和转换
# 注意：StandardScaler的输出是一个NumPy数组，而不是Pandas DataFrame
data_scaled = scaler.fit_transform(data)

# 为了方便查看，可以将其转换回DataFrame
data_scaled = pd.DataFrame(data_scaled, columns=data.columns)

print("\n-------------------------------------------------")
print("\n标准化后数据的统计信息:")
print(data_scaled.describe())

原始数据的统计信息:
             重量平均值       重量相对偏差       重量标准偏差       重量变异系数        重量最小值  \
count  1237.000000  1237.000000  1237.000000  1237.000000  1237.000000   
mean      0.512656     0.007065     0.010463     2.044025     0.490781   
std       0.004634     0.005635     0.001535     0.294834     0.006928   
min       0.501000     0.000087     0.005000     1.006000     0.466000   
25%       0.509500     0.002623     0.009000     1.838300     0.486000   
50%       0.512000     0.005744     0.010300     2.044000     0.491000   
75%       0.515200     0.010425     0.011200     2.222000     0.495000   
max       0.531000     0.035783     0.017100     3.347000     0.518000   

             重量最大值        吸阻平均值       吸阻相对偏差       吸阻标准偏差       吸阻变异系数  ...  \
count  1237.000000  1237.000000  1237.000000  1237.000000  1237.000000  ...   
mean      0.534255     1.330071     0.019747     0.049897     3.750157  ...   
std       0.007185     0.034055     0.016288     0.008702     0.636770  ...   
min   